In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp experiments

In [ ]:
# nothing

In [ ]:
#| export
import os, json, csv
from comet_ml import API, APIExperiment, start
from pct.hierarchy import PCTHierarchy  
from comet_ml.query import Metric

In [ ]:
#| export
class CometExperimentManager:
    def __init__(self, api_key: str = None, workspace: str = None):
        self.api = API(api_key)
        self.workspace = workspace

    def get_all_artifacts_indexed(self):
        """Retrieve all artifacts and sort them by source experiment key. Updates existing cache with new artifacts."""
        filename = '/tmp/artifacts/artifacts_results.json'
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        
        # Load existing results if file exists
        existing_results = {}
        if os.path.exists(filename):
            with open(filename, 'r') as file:
                existing_results = json.load(file)
        
        # Get all current artifacts from API
        artifacts = self.api.get_artifact_list(workspace=self.workspace)
        experiment = start(workspace=self.workspace)
        
        # Get current artifact names from API
        current_artifact_names = {artifact_dict['name'] for artifact_dict in artifacts['artifacts']}
        
        # Get existing artifact names from our cache
        existing_artifact_names = set(existing_results.values())
        
        # Find new artifacts that aren't in our cache
        new_artifact_names = current_artifact_names - existing_artifact_names
        
        results = existing_results.copy()  # Start with existing results
        
        # Process only new artifacts
        for artifact_dict in artifacts['artifacts']:
            artifact_name = artifact_dict['name']
            if artifact_name in new_artifact_names:
                try:
                    logged_artifact = experiment.get_artifact(artifact_name)
                    print(f"Adding new artifact: {logged_artifact.source_experiment_key} -> {artifact_name}")
                    results[logged_artifact.source_experiment_key] = artifact_name
                except Exception as e:
                    print(f"Error retrieving artifact {artifact_name}: {e}")

        # Save updated results to file
        with open(filename, "w") as file:
            json.dump(results, file, indent=4)
        experiment.end()
        
        if new_artifact_names:
            print(f"Added {len(new_artifact_names)} new artifacts to cache")
        else:
            print("No new artifacts found")
            
        return results

    def get_experiments_by_metrics(self, project_name: str = None, score_threshold: float = None, reward_threshold: float = None, max : bool = False):
        """
        Retrieve experiments for a project where the metric 'score' is less than
        score_threshold and 'reward_avg' is greater than or equal to reward_threshold.
        """
        if max:
            experiments = self.api.query(self.workspace, project_name, 
                                       (Metric("score") > score_threshold) & 
                                       (Metric("reward_avg") > reward_threshold))
        else:
            experiments = self.api.query(self.workspace, project_name, 
                                       (Metric("score") < score_threshold) & 
                                       (Metric("reward_avg") > reward_threshold))
 
        return experiments

    def get_artifact_name(self, experiment: APIExperiment = None):
        """Retrieve the name of an artifact from an experiment."""
        artifacts = experiment.get_artifacts()
        return artifacts[0]['artifact_name'] if artifacts else None

    def download_and_run_artifact(self, artifact_name: str = None, seeds: list[int] = None):
        """
        Download an artifact to '/tmp/artifacts/' and run PCTHierarchy.run_from_file
        with the artifact filename, returning the score value for each run.
        """
        download_path = f"/tmp/artifacts/"
        os.makedirs(os.path.dirname(download_path), exist_ok=True)
 
        full_path = os.path.join(download_path, artifact_name)
        if os.path.exists(full_path):
            pass
        else:
            experiment = start(workspace=self.workspace)
            logged_artifact  = experiment.get_artifact(artifact_name)
            # print(logged_artifact.source_experiment_key)
            # local_artifact = logged_artifact.download(download_path)
            logged_artifact.download(download_path)
            # filename = f"{local_artifact.download_local_path}{artifact_name}"
            experiment.end()
    
        rewards = []
        for seed in seeds:
            hierarchy, score = PCTHierarchy.run_from_file(full_path, seed=seed)
            metrics = hierarchy.get_environment().get_metrics()
            # print(f'Score={score:0.3f} {metrics}')
            rewards.append(metrics['reward'])



        return rewards

    def get_original_metrics(self, experiment: APIExperiment = None):
        """Retrieve the metrics for an experiment."""
        metrics = {}
        metrics['score'] = eval(experiment.get_metrics("score")[0]['metricValue'])
        hyperparameters = experiment.get_parameters_summary()

        for param in hyperparameters:
            if param['name'] == 'mode':
                metrics['mode'] = param['valueCurrent']
                break

        metrics['name'] = experiment.name
        return metrics

    def run_experiments_and_record_results(self, project_name: str = None, experiments: list[APIExperiment] = None, artifact_results: dict = None, num_runs: int = 0, output_csv: str = None):
        """
        Run each experiment for a given project a specified number of times and record the score and the
        number of times the reward is 100, -100, or something else. Save results to a CSV file.
        """
        results = []
        for experiment in experiments:
            try:
                artifact_name = artifact_results[experiment.id]
            except KeyError:
                print(f"WARNING: Artifact not found for experiment {experiment.id}")
                continue

            metrics = self.get_original_metrics(experiment)

            print(f"Running experiment {experiment.id} in project {project_name} with artifact {artifact_name}")
            if not artifact_name:
                continue

            rewards = self.download_and_run_artifact(artifact_name, seeds=range(num_runs))
            reward_counts = {'100': 0, '-100': 0, 'other': 0}

            for reward in rewards:
                if reward == 100:
                    reward_counts['100'] += 1
                elif reward == -100:
                    reward_counts['-100'] += 1
                else:
                    reward_counts['other'] += 1
            print(f"Rewards: {reward_counts}")
            results.append({
                'name': metrics['name'],
                'score': round(metrics['score'], 5),
                'mode': metrics['mode'],
                'reward_100': reward_counts['100'],
                'reward_-100': reward_counts['-100'],
                'reward_other': reward_counts['other'],
                'experiment_key': '/'.join(("https://www.comet.com", self.workspace, project_name, experiment.id)),
                'artifact_name': artifact_name
            })
        # Sort results by 'reward_100' in descending order, then by 'reward_other' in descending order
        results.sort(key=lambda x: (-x['reward_100'], -x['reward_other']))
        
        # Only save results to CSV if there are actually results
        if results:
            with open(output_csv, mode='w', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=['name', 'score', 'mode', 'reward_100', 'reward_-100', 'reward_other', 'experiment_key', 'artifact_name'])
                writer.writeheader()
                writer.writerows(results)
            print(f"Saved {len(results)} results to {output_csv}")
        else:
            print("No results to save - CSV file not created")

    def get_workspace_projects(self):
        """
        Get all projects from the current workspace.
        
        Returns:
            list: A list of project names in the workspace
        """
        api = API()
        projects = api.get_projects(workspace=self.workspace)
        return projects
        # return [project['name'] for project in projects]

In [ ]:
#|gui
# Initialize the manager
workspace = 'lunarlandercontinuous-v2'
project_name = 'refinputs-smooth'
manager = CometExperimentManager(workspace=workspace)

# Test get_all_artifacts_sorted
artifact_results = manager.get_all_artifacts_indexed()
# print("Artifacts sorted by source experiment key:", artifacts)

# Test get_experiments_by_metrics
experiments = manager.get_experiments_by_metrics(project_name=project_name, score_threshold=0.05, reward_threshold=10.0)
print("Filtered experiments:", experiments)



COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/lunarlandercontinuous-v2/general/01e2d222f6d44c56b93a45c409dbf07a

COMET INFO: Experiment is live on comet.com https://www.comet.com/lunarlandercontinuous-v2/general/01e2d222f6d44c56b93a45c409dbf07a

COMET INFO: The process of logging environment details (conda environment, git patch) is underway. Please be patient as this may take some time.
COMET INFO: The process of logging environment details (conda environment, git patch) is underway. Please be patient as this may take some time.


Error retrieving artifact ga-000.238-s002-3x1-m000-LL0001-d9aaf2358a62b99f9bbd6e7db60e631d.properties: Artifact {'consumer_experiment_key': '01e2d222f6d44c56b93a45c409dbf07a', 'experiment_key': '01e2d222f6d44c56b93a45c409dbf07a', 'name': 'ga-000.238-s002-3x1-m000-LL0001-d9aaf2358a62b99f9bbd6e7db60e631d.properties', 'version_or_alias': None, 'workspace': None} is not in a finalized state and cannot be accessed.


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : only_bee_5002
COMET INFO:     url                   : https://www.comet.com/lunarlandercontinuous-v2/general/01e2d222f6d44c56b93a45c409dbf07a
COMET INFO:   Uploads:
COMET INFO:     name                  : only_bee_5002
COMET INFO:     url                   : https://www.comet.com/lunarlandercontinuous-v2/general/01e2d222f6d44c56b93a45c409dbf07a
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 

Added 1 new artifacts to cache
Filtered experiments: []


In [ ]:
#|gui
# import random

# Test run_experiments_and_record_results
if experiments:
    output_csv = "/tmp/artifacts/experiment_results.csv"
    manager.run_experiments_and_record_results(experiments=experiments, project_name=project_name, artifact_results=artifact_results, num_runs=2, output_csv=output_csv)
    print(f"Results saved to {output_csv}")


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()